In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zoneinfo import ZoneInfo

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline

import sys
from pathlib import Path

project_dir = Path().resolve().parent
sys.path.append(str(project_dir))

# import my own modules
from commonfunc.db_handler import DbHandler
from strategies.univariate_strategy import MACDStrategy
from dataprocess.tech_indicator import TechnicalIndicators

#### Load Data

In [52]:
import os

# using absolute db path
db_file = os.path.join(project_dir, "data\crypto.db") 
db = DbHandler(db_file)

crypto = 'SOL'
crypto_lower = 'sol'

# query price data
query = f"SELECT * FROM {crypto_lower}_hourly order by time" 
df = db.read_from_db(query)

# convert dt columns types to datetime
df['utc_dt'] = pd.to_datetime(df['utc_dt'])
df['nyc_dt'] = df['utc_dt'].apply(lambda x: x.astimezone(ZoneInfo("America/New_York")))

# only use data after 2024
df = df[df['utc_dt']>='2024-01-01']

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9975 entries, 26280 to 36254
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype                           
---  ------      --------------  -----                           
 0   time        9975 non-null   int64                           
 1   utc_dt      9975 non-null   datetime64[ns, UTC]             
 2   nyc_dt      9975 non-null   datetime64[ns, America/New_York]
 3   high        9975 non-null   float64                         
 4   low         9975 non-null   float64                         
 5   open        9975 non-null   float64                         
 6   close       9975 non-null   float64                         
 7   volumefrom  9975 non-null   float64                         
 8   volumeto    9975 non-null   float64                         
 9   avg         9975 non-null   float64                         
dtypes: datetime64[ns, America/New_York](1), datetime64[ns, UTC](1), float64(7), int64(1)
memory

#### Compute Technical Metrics

In [53]:
ti = TechnicalIndicators(df.copy())
df = ti.compute_all_indicators()
df.tail()

,time,utc_dt,nyc_dt,high,low,open,close,volumefrom,volumeto,avg,sma_24,sma_72,ema_24,ema_72,bollinger_upper_24,bollinger_lower_24,atr_14,rsi_14,macd,macd_signal,stoch_osc_14,stoch_rsi_14,cci_20,adx_14,plus_di_14,minus_di_14
36250,1739959200,2025-02-19 10:00:00+00:00,2025-02-19 05:00:00-05:00,172.40,170.42,170.75,172.08,76242.21,13080932.42,171.570740,167.485833,177.265278,169.090111,175.949433,172.555638,162.416029,2.495714,67.513863,-0.487843,-1.445601,96.304850,100.000000,94.224713,5.294588,26.932989,24.224415
36251,1739962800,2025-02-19 11:00:00+00:00,2025-02-19 06:00:00-05:00,174.24,171.99,172.08,173.89,80407.89,13950988.78,173.502734,167.702917,176.989444,169.474102,175.893010,173.394223,162.011610,2.328571,65.623472,-0.046267,-1.165735,96.666667,92.331046,124.945288,14.745788,32.448515,24.108707
36252,1739966400,2025-02-19 12:00:00+00:00,2025-02-19 07:00:00-05:00,174.08,171.64,173.89,171.65,63457.84,10973135.20,172.920087,167.743333,176.697639,169.648174,175.776763,173.535762,161.950904,2.309286,58.763345,0.121534,-0.908281,75.333333,64.500832,101.934042,12.447021,30.382397,23.656201
36253,1739970000,2025-02-19 13:00:00+00:00,2025-02-19 08:00:00-05:00,172.68,170.89,171.65,171.56,70872.95,12181366.56,171.876104,167.822917,176.430972,169.801120,175.661235,173.774970,161.870863,2.280714,55.534442,0.244438,-0.677737,74.476190,51.401792,81.766514,7.478165,28.565652,24.590544
36254,1739973600,2025-02-19 14:00:00+00:00,2025-02-19 09:00:00-05:00,171.70,169.92,171.56,170.05,34916.55,5960040.99,170.693868,167.953333,176.151806,169.821030,175.507503,173.959726,161.946941,2.239286,53.233151,0.217489,-0.498692,60.095238,42.065899,49.092253,1.246777,27.015986,26.350624


#### Instantiate and Generate signals with a strategy

In [54]:
macd_strategy = MACDStrategy(data = df)
df = macd_strategy.generate_signals()
df.tail()

,time,utc_dt,nyc_dt,high,low,open,close,volumefrom,volumeto,avg,sma_24,sma_72,ema_24,ema_72,bollinger_upper_24,bollinger_lower_24,atr_14,rsi_14,macd,macd_signal,stoch_osc_14,stoch_rsi_14,cci_20,adx_14,plus_di_14,minus_di_14,signal
36250,1739959200,2025-02-19 10:00:00+00:00,2025-02-19 05:00:00-05:00,172.40,170.42,170.75,172.08,76242.21,13080932.42,171.570740,167.485833,177.265278,169.090111,175.949433,172.555638,162.416029,2.495714,67.513863,-0.487843,-1.445601,96.304850,100.000000,94.224713,5.294588,26.932989,24.224415,1
36251,1739962800,2025-02-19 11:00:00+00:00,2025-02-19 06:00:00-05:00,174.24,171.99,172.08,173.89,80407.89,13950988.78,173.502734,167.702917,176.989444,169.474102,175.893010,173.394223,162.011610,2.328571,65.623472,-0.046267,-1.165735,96.666667,92.331046,124.945288,14.745788,32.448515,24.108707,1
36252,1739966400,2025-02-19 12:00:00+00:00,2025-02-19 07:00:00-05:00,174.08,171.64,173.89,171.65,63457.84,10973135.20,172.920087,167.743333,176.697639,169.648174,175.776763,173.535762,161.950904,2.309286,58.763345,0.121534,-0.908281,75.333333,64.500832,101.934042,12.447021,30.382397,23.656201,1
36253,1739970000,2025-02-19 13:00:00+00:00,2025-02-19 08:00:00-05:00,172.68,170.89,171.65,171.56,70872.95,12181366.56,171.876104,167.822917,176.430972,169.801120,175.661235,173.774970,161.870863,2.280714,55.534442,0.244438,-0.677737,74.476190,51.401792,81.766514,7.478165,28.565652,24.590544,1
36254,1739973600,2025-02-19 14:00:00+00:00,2025-02-19 09:00:00-05:00,171.70,169.92,171.56,170.05,34916.55,5960040.99,170.693868,167.953333,176.151806,169.821030,175.507503,173.959726,161.946941,2.239286,53.233151,0.217489,-0.498692,60.095238,42.065899,49.092253,1.246777,27.015986,26.350624,1


#### Transform Data

In [55]:
df = df.rename(columns={
    'utc_dt': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volumefrom': 'Volume'
})

# Ensure Date is a pandas datetime and set as index (not strictly required, but typical)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

#### Define a Strategy that Uses the "signal"

In [63]:
from backtesting import Strategy

class ExternalSignalStrategy(Strategy):
    """
    Strategy that reads a 'signal' column from the DataFrame:
      - +1 = buy/long
      - -1 = sell/exit (or go short)
      - 0 = do nothing
    """
    def init(self):
        # We'll store the entire signal series so we can read it in next().
        # self.data is a backtesting._Data instance, but we can access df via .df or by indexing.
        self.signals = self.data.df['signal']

    def next(self):
        # current bar index is self.bar_index
        current_signal = self.signals[-1]

        # If signal == 1 and we have no open position, buy
        if current_signal == 1 and not self.position:
            self.buy()
        
        # If signal == -1 and we are in a long position, sell/exit
        elif current_signal == -1 and self.position.is_long:
            self.exit()

        # If your signals also included shorting logic, you'd handle that similarly:
        # if current_signal == -1 and not self.position:
        #     self.sell()  # opens a short position

        # If signal == 0, do nothing


#### Run Backtest

In [65]:
from backtesting import Backtest

# Suppose df is your renamed DataFrame with columns: Open, High, Low, Close, Volume, signal
# df index: Date
# ... set up any final config ...
t1 = '2024-03-01'
t2 = '2024-03-15'
df_test = df[(df.index >= t1)&(df.index <= t2)]

bt = Backtest(
    df_test,                    # your price + signal data
    ExternalSignalStrategy,
    cash=10000,            # initial capital
    commission=0.001,      # e.g. 0.1% commission
    trade_on_close=True    # optional
)

stats = bt.run()
print(stats)
bt.plot(plot_pl=True,plot_drawdown=True)


Start                     2024-03-01 00:00...
End                       2024-03-15 00:00...
Duration                     14 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  13179.46025
Equity Peak [$]                   13179.46025
Return [%]                            31.7946
Buy & Hold Return [%]                34.91729
Return (Ann.) [%]                263152.47357
Volatility (Ann.) [%]            236282.30038
CAGR [%]                          133529.5036
Sharpe Ratio                          1.11372
Sortino Ratio                      8889.72875
Calmar Ratio                      17737.93464
Max. Drawdown [%]                   -14.83558
Avg. Drawdown [%]                    -4.35087
Max. Drawdown Duration        4 days 11:00:00
Avg. Drawdown Duration        1 days 01:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                   

Row(id='7278', ...)